In [1]:
import os
import numpy as np
import pandas as pd
from pathlib import Path
from datetime import datetime
from preprocessing import *

In [2]:
# 환경설정
pd.options.display.float_format = '{:,.0f}'.format
os.makedirs('result', exist_ok=True)

# 전역변수
FILE_PATH = Path('data/현행추정부채_보험가격준비금익스포져_일반/20210727')
BASE_YYMM = '201912'

In [3]:
# 메모 (2021.07.25)
# 일반 최종손해율 export 할 때, sql RRNR_DVCD로 바꾸기
# 익스포져 외 데이터 불러올 때 dtype 명시 다시 하기
# 일반_사업비율 04 -> 01, 02 처리 현재는 수기로 하고 있음

In [9]:
# 데이터 불러오기
## 보험료부채 익스포져
일반_원수_미경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_미경과보험료_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'INRI_RN_KND_CD': str, 'CMSN_DVCD': str, 'OGL_URND_PRM': float})
일반_출재_미경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_미경과보험료_{BASE_YYMM}.xlsx',
        dtype={'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'RN_KND_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_URND_PRM': float})
일반_원수_예정보험료 = pd.read_excel(FILE_PATH / f'일반_원수_예정보험료_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'OGL_EPCT_PRM': float})
일반_출재_예정보험료 = pd.read_excel(FILE_PATH / f'일반_출재_예정보험료_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'T02_RN_RINSC_CD': str, 'RN_EPCT_PRM': float})
일반_원수_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'일반_원수_직전3년보험금손조비_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'OGL_PYN_BNF': float, 'OGL_ACCD_IVMT': float})
일반_출재_직전3년보험금손조비 = pd.read_excel(FILE_PATH / f'일반_출재_직전3년보험금손조비_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'RN_PYN_BNF': float, 'RN_ACCD_IVMT': float})
# 일반_원수_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_원수_직전1년경과보험료_{BASE_YYMM}.xlsx',
#       dtype={'RRNR_DAT_DVCD': str, 'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'INRI_RN_KND_CD': str, 'CMSN_DVCD': str, 'OGL_ELP_PRM': float})
일반_출재_직전1년경과보험료 = pd.read_excel(FILE_PATH / f'일반_출재_직전1년경과보험료_{BASE_YYMM}.xlsx',
        dtype={'RRNR_CTC_BZ_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'RN_KND_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_ELP_PRM': float})

## 준비금부채 익스포져
일반_원수_개별추산액 = pd.read_excel(FILE_PATH / f'일반_원수_개별추산액_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'INRI_RN_KND_CD': str, 'CMSN_DVCD': str, 'OGL_OST_AMT': float})
일반_출재_개별추산액 = pd.read_excel(FILE_PATH / f'일반_출재_개별추산액_{BASE_YYMM}.xlsx',
        dtype={'RRNR_DMFR_DVCD': str, 'PDC_CD': str, 'INER_CHN_DVCD': str, 'NTNL_CTRY_CD': str, 'RN_KND_CD': str, 'T02_RN_RINSC_CD': str, 'CMSN_DVCD': str, 'RN_OST_AMT': float})

## 기초정보, 가정 등
일반_상품정보 = pd.read_excel(FILE_PATH / '일반_상품정보.xlsx',
        dtype={'PDC_CD': str, 'PDC_NM': str, 'PDGR_CD': str, 'PDGR_NM': str})
일반_보험금진전추이 = pd.read_excel(FILE_PATH / f'일반_보험금진전추이_{202012}.xlsx',
        dtype={'PDGR_CD': str, 'AY': str, 'BASE_1': float, 'BASE_2': float, 'BASE_3': float, 'BASE_4': float, 'BASE_5': float, 'BASE_6': float, 'BASE_7': float})
일반_최종손해율 = pd.read_excel(FILE_PATH / f'일반_최종손해율_{202012}.xlsx',
        dtype={'RRNR_DAT_DVCD': str, 'PDGR_CD': str}) \
        .rename(columns= {'RRNR_DAT_DVCD': 'RRNR_DVCD'})
일반_사업비율 = pd.read_excel(FILE_PATH / f'일반_사업비율_{202012}.xlsx', dtype={'RRNR_DVCD': str, 'DMFR_DVCD': str}) \
        .assign(BOZ_CD = lambda x: x['BOZ_CD'].str.replace('A100', '#'))
일반_IBNR = pd.read_excel(FILE_PATH / f'일반_IBNR_{202012}.xlsx', dtype={'RRNR_DVCD': str, 'IBNR_KEY': str}) \
        .drop('CLG_YM', axis=1) \
        .assign(RRNR_DVCD = lambda x: x['RRNR_DVCD'].str.pad(width=2))

## 공통
국가그룹 = pd.read_excel(FILE_PATH / '국가그룹.xlsx', dtype={'CNTR_CATG_CD': str})
재보험자_국내신용등급 = pd.read_excel(FILE_PATH / f'재보험자_국내신용등급_{202012}.xlsx', dtype={'RINSC_CD': str, 'CRD_GRD': str})
재보험자_KICS신용등급 = pd.read_excel(FILE_PATH / '재보험자_KICS신용등급_202012.xlsx', dtype={'RINSC_CD': str, 'KICS_CRD_GRD': str})
재보험자_수정 = pd.read_excel(FILE_PATH / '재보험자_수정_202012.xlsx', dtype={'T02_RN_RINSC_CD': str, 'RINSC_CD': str})
선도부도율 = pd.read_excel(FILE_PATH / f'선도부도율_{202012}.xlsx')
할인율 = pd.read_excel(FILE_PATH / f'할인율_{202012}.xlsx')

In [10]:
# 데이터 전처리
## 미경과보험료
## (2021.07.25) INER_CHN_DVCD 처리 로직 추가하기
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료.copy()
일반_원수_미경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_미경과보험료_가공)
일반_원수_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_미경과보험료_가공)
일반_원수_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_미경과보험료_가공, 국가그룹)
일반_원수_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_미경과보험료_가공, 일반_상품정보)
일반_원수_미경과보험료_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_원수_미경과보험료_가공)
일반_원수_미경과보험료_가공['RINSC_CD'] = '#'
일반_원수_미경과보험료_가공['CRD_GRD'] = '#'
일반_원수_미경과보험료_가공['KICS_CRD_GRD'] = '#'
일반_원수_미경과보험료_가공.rename(columns={'OGL_URND_PRM': 'URND_PRM'}, inplace=True)
일반_원수_미경과보험료_가공 = 일반_원수_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CNTR_CATG_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'URND_PRM']]

일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료.copy()
일반_출재_미경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_미경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_미경과보험료_가공)
일반_출재_미경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_미경과보험료_가공, 국가그룹)
일반_출재_미경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_미경과보험료_가공, 일반_상품정보)
일반_출재_미경과보험료_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_출재_미경과보험료_가공)
일반_출재_미경과보험료_가공['RINSC_CD'] = clsf_rinsc_cd(일반_출재_미경과보험료_가공, 재보험자_수정)
일반_출재_미경과보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_미경과보험료_가공, 재보험자_국내신용등급)
일반_출재_미경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(일반_출재_미경과보험료_가공, 재보험자_KICS신용등급)
일반_출재_미경과보험료_가공.rename(columns={'RN_URND_PRM': 'URND_PRM'}, inplace=True)
일반_출재_미경과보험료_가공 = 일반_출재_미경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CNTR_CATG_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'URND_PRM']]

일반_미경과보험료_가공 = pd.concat([일반_원수_미경과보험료_가공, 일반_출재_미경과보험료_가공], axis=0).reset_index(drop=True)

## 예정보험료
일반_원수_예정보험료_가공 = 일반_원수_예정보험료.copy()
일반_원수_예정보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_예정보험료_가공)
일반_원수_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_예정보험료_가공)
일반_원수_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_예정보험료_가공, 일반_상품정보)
일반_원수_예정보험료_가공['RINSC_CD'] = '#'
일반_원수_예정보험료_가공['CRD_GRD'] = '#'
일반_원수_예정보험료_가공['KICS_CRD_GRD'] = '#'
일반_원수_예정보험료_가공.rename(columns = {'OGL_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
일반_원수_예정보험료_가공 = 일반_원수_예정보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'EPCT_PRM']]

일반_출재_예정보험료_가공 = 일반_출재_예정보험료.copy()
일반_출재_예정보험료_가공['RRNR_DVCD'] = '03'
일반_출재_예정보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_예정보험료_가공)
일반_출재_예정보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_예정보험료_가공, 일반_상품정보)
일반_출재_예정보험료_가공['RINSC_CD'] = clsf_rinsc_cd(일반_출재_예정보험료_가공, 재보험자_수정)
일반_출재_예정보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_예정보험료_가공, 재보험자_국내신용등급)
일반_출재_예정보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(일반_출재_예정보험료_가공, 재보험자_KICS신용등급)

일반_출재_예정보험료_가공.rename(columns = {'RN_EPCT_PRM': 'EPCT_PRM'}, inplace=True)
일반_출재_예정보험료_가공 = 일반_출재_예정보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'EPCT_PRM']]

일반_예정보험료_가공 = pd.concat([일반_원수_예정보험료_가공, 일반_출재_예정보험료_가공], axis=0).reset_index(drop=True)

## 직전1년경과보험료
# 일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료.copy()
# 일반_원수_직전1년경과보험료_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전1년경과보험료_가공)
# 일반_원수_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전1년경과보험료_가공)
# 일반_원수_직전1년경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_직전1년경과보험료_가공, 국가그룹)
# 일반_원수_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_직전1년경과보험료_가공, 일반_상품정보)
# 일반_원수_직전1년경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_직전1년경과보험료_가공, 일반_상품정보)
# 일반_원수_직전1년경과보험료_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_원수_직전1년경과보험료_가공)
# 일반_원수_직전1년경과보험료_가공['CRD_GRD'] = '#'
# 일반_원수_직전1년경과보험료_가공.rename(columns = {'OGL_ELP_PRM': 'ELP_PRM'}, inplace=True)
# 일반_원수_직전1년경과보험료_가공 = 일반_원수_직전1년경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CRD_GRD', 'ELP_PRM']]

일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료.copy()
일반_출재_직전1년경과보험료_가공['RRNR_DVCD'] = '03'
일반_출재_직전1년경과보험료_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전1년경과보험료_가공)
일반_출재_직전1년경과보험료_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_직전1년경과보험료_가공, 국가그룹)
일반_출재_직전1년경과보험료_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_직전1년경과보험료_가공, 일반_상품정보)
일반_출재_직전1년경과보험료_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_직전1년경과보험료_가공, 일반_상품정보)
일반_출재_직전1년경과보험료_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_출재_직전1년경과보험료_가공)
일반_출재_직전1년경과보험료_가공['RINSC_CD'] = clsf_rinsc_cd(일반_출재_직전1년경과보험료_가공, 재보험자_수정)
일반_출재_직전1년경과보험료_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_직전1년경과보험료_가공, 재보험자_국내신용등급)
일반_출재_직전1년경과보험료_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(일반_출재_직전1년경과보험료_가공, 재보험자_KICS신용등급)
일반_출재_직전1년경과보험료_가공.rename(columns = {'RN_ELP_PRM': 'ELP_PRM'}, inplace=True)
일반_출재_직전1년경과보험료_가공 = 일반_출재_직전1년경과보험료_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CNTR_CATG_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'ELP_PRM']]

# 일반_직전1년경과보험료_가공 = pd.concat([일반_원수_직전1년경과보험료_가공, 일반_출재_직전1년경과보험료_가공], axis=0).reset_index(drop=True)

## 개별추산액
일반_원수_개별추산액_가공 = 일반_원수_개별추산액.copy()
일반_원수_개별추산액_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_개별추산액_가공)
일반_원수_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_개별추산액_가공)
일반_원수_개별추산액_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_원수_개별추산액_가공, 국가그룹)
일반_원수_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_개별추산액_가공, 일반_상품정보)
일반_원수_개별추산액_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_원수_개별추산액_가공)
일반_원수_개별추산액_가공['RINSC_CD'] = '#'
일반_원수_개별추산액_가공['CRD_GRD'] = '#'
일반_원수_개별추산액_가공['KICS_CRD_GRD'] = '#'
일반_원수_개별추산액_가공.rename(columns = {'OGL_OST_AMT': 'OST_AMT'}, inplace=True)
일반_원수_개별추산액_가공 = 일반_원수_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CNTR_CATG_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'OST_AMT']]

일반_출재_개별추산액_가공 = 일반_출재_개별추산액.copy()
일반_출재_개별추산액_가공['RRNR_DVCD'] = '03'
일반_출재_개별추산액_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_개별추산액_가공)
일반_출재_개별추산액_가공['CNTR_CATG_CD'] = clsf_cntr_catg_cd(일반_출재_개별추산액_가공, 국가그룹)
일반_출재_개별추산액_가공['BOZ_CD'] = clsf_boz_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_개별추산액_가공, 일반_상품정보)
일반_출재_개별추산액_가공['P_NP_DVCD'] = clsf_p_np_dvcd(일반_출재_개별추산액_가공)
일반_출재_개별추산액_가공['RINSC_CD'] = clsf_rinsc_cd(일반_출재_개별추산액_가공, 재보험자_수정)
일반_출재_개별추산액_가공['CRD_GRD'] = clsf_crd_grd(일반_출재_개별추산액_가공, 재보험자_국내신용등급)
일반_출재_개별추산액_가공['KICS_CRD_GRD'] = clsf_kics_crd_grd(일반_출재_개별추산액_가공, 재보험자_KICS신용등급)
일반_출재_개별추산액_가공.rename(columns = {'RN_OST_AMT': 'OST_AMT'}, inplace=True)
일반_출재_개별추산액_가공 = 일반_출재_개별추산액_가공[['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD', 'PDGR_CD', 'CNTR_CATG_CD', 'P_NP_DVCD', 'CMSN_DVCD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'OST_AMT']]

일반_개별추산액_가공 = pd.concat([일반_원수_개별추산액_가공, 일반_출재_개별추산액_가공], axis=0).reset_index(drop=True)

## 직전3년보험금손조비
일반_원수_직전3년보험금손조비_가공 = 일반_원수_직전3년보험금손조비.copy()
일반_원수_직전3년보험금손조비_가공['RRNR_DVCD'] = clsf_rrnr_dvcd(일반_원수_직전3년보험금손조비_가공)
일반_원수_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_원수_직전3년보험금손조비_가공)
일반_원수_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_원수_직전3년보험금손조비_가공, 일반_상품정보)
일반_원수_직전3년보험금손조비_가공.rename(columns = {'OGL_PYN_BNF': 'PYN_BNF', 'OGL_ACCD_IVMT': 'ACCD_IVMT'}, inplace=True)
일반_원수_직전3년보험금손조비_가공 = 일반_원수_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'PYN_BNF', 'ACCD_IVMT']]

일반_출재_직전3년보험금손조비_가공 = 일반_출재_직전3년보험금손조비.copy()
일반_출재_직전3년보험금손조비_가공['RRNR_DVCD'] = '03'
일반_출재_직전3년보험금손조비_가공['DMFR_DVCD'] = clsf_dmfr_dvcd(일반_출재_직전3년보험금손조비_가공)
일반_출재_직전3년보험금손조비_가공['PDGR_CD'] = clsf_pdgr_cd(일반_출재_직전3년보험금손조비_가공, 일반_상품정보)
일반_출재_직전3년보험금손조비_가공.rename(columns = {'RN_PYN_BNF': 'PYN_BNF', 'RN_ACCD_IVMT': 'ACCD_IVMT'}, inplace=True)
일반_출재_직전3년보험금손조비_가공 = 일반_출재_직전3년보험금손조비_가공[['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'PYN_BNF', 'ACCD_IVMT']]

일반_직전3년보험금손조비_가공 = pd.concat([일반_원수_직전3년보험금손조비_가공, 일반_출재_직전3년보험금손조비_가공], axis=0).reset_index(drop=True)

In [42]:
pd.options.display.float_format = '{:,.10f}'.format

# 가정산출
## 최종손해율
### 8개 상품군(화재, 기술, 근재, 책임, 종합, 상해, 기타, 해상) 가정 사용
일반_최종손해율 = 일반_최종손해율.loc[lambda x: x['PDGR_CD'].isin(['23', '24', '25', '26', '27', '28', '29', '31'])].reset_index(drop=True)

## 손해조사비율
### 1. 원수+수재, 출재로 그룹핑
### 2. PDGR_CD 단위로 그룹핑
### 3. 해외의 경우 PDGR_CD 구분없이 산출
일반_손해조사비율 = 일반_직전3년보험금손조비_가공.copy()
일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02', 'PDGR_CD'] = '#'
일반_손해조사비율 = 일반_손해조사비율.groupby(['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'])[['ACCD_IVMT', 'PYN_BNF']].sum() \
        .eval('LAE_RATIO = ACCD_IVMT/PYN_BNF') \
        .reset_index() \
        .drop(['ACCD_IVMT', 'PYN_BNF'], axis=1)
일반_손해조사비율_국내 = 일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='01']
일반_손해조사비율_해외 = pd.concat([일반_손해조사비율.loc[lambda x: x['DMFR_DVCD']=='02'].copy().assign(PDGR_CD = pdgr_cd) for pdgr_cd in ['23', '24', '25', '26', '27', '28', '29', '31']])
일반_손해조사비율 = pd.concat([일반_손해조사비율_국내, 일반_손해조사비율_해외], axis=0)
일반_원수_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '01')
일반_수재_손해조사비율 = 일반_손해조사비율.loc[lambda x: x['RRNR_DVCD'] == '04'].copy().assign(RRNR_DVCD = '02')
일반_손해조사비율 = pd.concat([일반_손해조사비율, 일반_원수_손해조사비율, 일반_수재_손해조사비율]).reset_index(drop=True)

## 할인요소
### (2021.07.25) KICS_SCEN_NO 컬럼 추가
일반_할인요소 = []
for 시나리오 in 할인율['KICS_SCEN_NO'].unique():
    일반_할인요소_개별시나리오 = get_disc_factor_all(일반_보험금진전추이, 할인율.query('KICS_SCEN_NO == @시나리오'))
    일반_할인요소_개별시나리오.insert(0, 'KICS_SCEN_NO', 시나리오)
    일반_할인요소.append(일반_할인요소_개별시나리오)
일반_할인요소 = pd.concat(일반_할인요소, axis=0)

## 재보험자산 손실조정율
### (2021.07.25) KICS_SCEN_NO 컬럼 추가
일반_손실조정율 = []
for 시나리오 in 할인율['KICS_SCEN_NO'].unique():
    일반_손실조정율_개별시나리오 = get_loss_adj_rate_all(일반_보험금진전추이, 할인율.query('KICS_SCEN_NO == @시나리오'), 선도부도율)
    일반_손실조정율_개별시나리오.insert(0, 'KICS_SCEN_NO', 시나리오)
    일반_손실조정율.append(일반_손실조정율_개별시나리오)
일반_손실조정율 = pd.concat(일반_손실조정율, axis=0)

## IBNR
### DMFR_DVCD, PDGR_DVCD, RRNR_DVCD(원수+수재는 묶어서) 단위로 배분
일반_IBNR_가공 = 일반_IBNR.copy()
일반_IBNR_가공 = 일반_IBNR_가공.assign(
    PDGR_CD = lambda x: x['IBNR_KEY'].str.split('_').map(lambda x: x[1] if len(x)==2 else '#'),
    DMFR_DVCD = lambda x: x['IBNR_KEY'].str.split('_').map(lambda x: f'{int(x[0]):02}')
)
일반_IBNR_가공.rename(columns = {'KRW_OS_AMT_E': 'IBNR_TOT'}, inplace=True)
일반_IBNR_가공.drop('IBNR_KEY', axis=1, inplace=True)
일반_IBNR_가공 = 일반_IBNR_가공.query('RRNR_DVCD != "02"') \
    .assign(RRNR_DVCD = lambda x: np.where(x['RRNR_DVCD']=='01', '04', x['RRNR_DVCD']))

## 사업비율
일반_사업비율_가공 = 일반_사업비율.copy()
for rrnr_dvcd in ['01', '02', '03']:
    for boz_cd in ['A001', 'A002', 'A003', 'A004', 'A005', 'A006', 'A007', 'A008', 'A009', 'A010', 'A011']:
        일반_사업비율_가공 = 일반_사업비율_가공.append({'RRNR_DVCD': rrnr_dvcd, 'DMFR_DVCD': '02', 'BOZ_CD': boz_cd, 'EXP_RT': 일반_사업비율.query('RRNR_DVCD == @rrnr_dvcd and DMFR_DVCD == "02"')['EXP_RT'].values[0]}, ignore_index=True)
일반_사업비율_가공.query('BOZ_CD != "#"', inplace=True)

## 유효만기
일반_유효만기 = []
for pdgr_cd in 일반_보험금진전추이['PDGR_CD'].unique():
    cf_t_prm, cf_rate_prm = get_cf(일반_보험금진전추이.query('PDGR_CD == @pdgr_cd'), pdgr_cd, '보험료')
    cf_t_rsv, cf_rate_rsv = get_cf(일반_보험금진전추이.query('PDGR_CD == @pdgr_cd'), pdgr_cd, '보험금')
    일반_유효만기.append([pdgr_cd, sum(cf_t_prm*cf_rate_prm), sum(cf_t_rsv*cf_rate_rsv)])
일반_유효만기 = pd.DataFrame(일반_유효만기, columns=['PDGR_CD', 'EFF_MAT_PRM', 'EFF_MAT_RSV'])

In [110]:
PV_LIAB_PRM

NameError: name 'PV_LIAB_PRM' is not defined

In [115]:

일반_보험료부채

,RRNR_DVCD,DMFR_DVCD,PDGR_CD,RINSC_CD,CRD_GRD,KICS_CRD_GRD,BOZ_CD,URND_PRM,EPCT_PRM,FNAL_LSRT,...,MAINT_EXP,FNAL_LOSS,LAE,LIAB_PRM,PV_EPCT_PRM,PV_FNAL_LOSS,PV_LAE,PV_MAINT_EXP,PV_LIAB_PRM,LOSS_ADJ_PRM
0,04,01,23,#,#,#,A001,"23,420,916,171",0,1,...,"1,185,224,492","12,866,069,510","648,573,166","14,699,867,169",0,"12,824,338,198","646,469,508","1,181,380,196","14,652,187,903",0
1,04,01,23,#,#,#,A001,"23,420,916,171",0,1,...,"1,185,224,492","12,866,069,510","648,573,166","14,699,867,169",0,"12,823,936,563","646,449,262","1,181,343,198","14,651,729,023",0
2,04,01,23,#,#,#,A001,"23,420,916,171",0,1,...,"1,185,224,492","12,866,069,510","648,573,166","14,699,867,169",0,"12,812,042,844","645,849,705","1,180,247,546","14,638,140,095",0
3,04,01,23,#,#,#,A001,"23,420,916,171",0,1,...,"1,185,224,492","12,866,069,510","648,573,166","14,699,867,169",0,"12,836,665,034","647,090,899","1,182,515,747","14,666,271,680",0
4,04,01,23,#,#,#,A001,"23,420,916,171",0,1,...,"1,185,224,492","12,866,069,510","648,573,166","14,699,867,169",0,"12,823,747,173","646,439,715","1,181,325,751","14,651,512,639",0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4189,03,02,31,124012,AA,3,#,"78,701,115",0,1,...,"2,990,114","54,387,743","4,114,494","61,492,352",0,"53,996,385","4,084,887","2,968,598","61,049,871","27,644"
4190,03,02,31,124012,AA,3,#,"78,701,115",0,1,...,"2,990,114","54,387,743","4,114,494","61,492,352",0,"53,844,483","4,073,396","2,960,247","60,878,126","27,615"
4191,03,02,31,124012,AA,3,#,"78,701,115",0,1,...,"2,990,114","54,387,743","4,114,494","61,492,352",0,"54,167,599","4,097,840","2,978,011","61,243,450","27,676"
4192,03,02,31,124012,AA,3,#,"78,701,115",0,1,...,"2,990,114","54,387,743","4,114,494","61,492,352",0,"53,965,404","4,082,544","2,966,895","61,014,843","27,643"


,RINSC_CD,KICS_CRD_GRD,PV_LIAB_PRM_LOSS_ADJ
0,121002,3,"4,797,491"
1,121005,3,"162,564"
2,121017,3,"51,085,148"
3,121018,3,"98,785,129"
4,121028,3,"3,419,153"
...,...,...,...
280,999A11,2,"470,489,860"
281,999A12,2,"5,431"
282,999A21,3,"1,187,391,691"
283,999A22,3,"15,396,019"


In [134]:

## 보험료부채
일반_할인전보험료부채유효만기_재보험자별 = 일반_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_PRM']*x['EFF_MAT_PRM'])).reset_index(name='LIAB_PRM_MULT_EFF_MAT') \
    .query('LIAB_PRM_MULT_EFF_MAT != 0')
일반_할인전보험료부채_재보험자별 = 일반_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_PRM'])).reset_index(name='LIAB_PRM') \
    .query('LIAB_PRM != 0')
일반_보험료부채_재보험자별 = 일반_보험료부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['PV_LIAB_PRM']-x['LOSS_ADJ_PRM'])).reset_index(name='PV_LIAB_PRM_LOSS_ADJ') \
    .query('PV_LIAB_PRM_LOSS_ADJ != 0')
일반_유효만기_보험료부채_재보험자별 = 일반_할인전보험료부채_재보험자별 \
    .merge(일반_보험료부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .merge(일반_할인전보험료부채유효만기_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .eval('EFF_MAT_PRM = LIAB_PRM_MULT_EFF_MAT/LIAB_PRM') \
    .drop(['LIAB_PRM_MULT_EFF_MAT'], axis=1)

## 준비금부채
일반_할인전준비금부채유효만기_재보험자별 = 일반_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_RSV']*x['EFF_MAT_RSV'])).reset_index(name='LIAB_RSV_MULT_EFF_MAT') \
    .query('LIAB_RSV_MULT_EFF_MAT != 0')
일반_할인전준비금부채_재보험자별 = 일반_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['LIAB_RSV'])).reset_index(name='LIAB_RSV') \
    .query('LIAB_RSV != 0')
일반_준비금부채_재보험자별 = 일반_준비금부채.query('KICS_SCEN_NO == 1').query('RRNR_DVCD == "03"') \
    .groupby(['RINSC_CD', 'KICS_CRD_GRD']).apply(lambda x: sum(x['PV_LIAB_RSV']-x['LOSS_ADJ_RSV'])).reset_index(name='PV_LIAB_RSV_LOSS_ADJ') \
    .query('PV_LIAB_RSV_LOSS_ADJ != 0')
일반_유효만기_준비금부채_재보험자별 = 일반_할인전준비금부채_재보험자별 \
    .merge(일반_준비금부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .merge(일반_할인전준비금부채유효만기_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer') \
    .eval('EFF_MAT_RSV = LIAB_RSV_MULT_EFF_MAT/LIAB_RSV') \
    .drop(['LIAB_RSV_MULT_EFF_MAT'], axis=1)

일반_유효만기_재보험자별 = 일반_유효만기_보험료부채_재보험자별 \
    .merge(일반_유효만기_준비금부채_재보험자별, on=['RINSC_CD', 'KICS_CRD_GRD'], how='outer')
일반_유효만기_재보험자별[['LIAB_PRM', 'EFF_MAT_PRM', 'LIAB_RSV', 'EFF_MAT_RSV', 'PV_LIAB_PRM_LOSS_ADJ', 'PV_LIAB_RSV_LOSS_ADJ']] = 일반_유효만기_재보험자별[['LIAB_PRM', 'EFF_MAT_PRM', 'LIAB_RSV', 'EFF_MAT_RSV', 'PV_LIAB_PRM_LOSS_ADJ', 'PV_LIAB_RSV_LOSS_ADJ']].fillna(0)
일반_유효만기_재보험자별.eval('EFF_MAT = (EFF_MAT_PRM*LIAB_PRM+EFF_MAT_RSV*LIAB_RSV)/(LIAB_PRM+LIAB_RSV)', inplace=True)
일반_유효만기_재보험자별.eval('PV_LIAB_LOSS_ADJ = PV_LIAB_PRM_LOSS_ADJ+PV_LIAB_RSV_LOSS_ADJ', inplace=True)
일반_유효만기_재보험자별.eval('LIAB = LIAB_PRM+LIAB_RSV', inplace=True)
일반_유효만기_재보험자별 = 일반_유효만기_재보험자별[['RINSC_CD', 'KICS_CRD_GRD', 'EFF_MAT', 'LIAB', 'PV_LIAB_LOSS_ADJ']]
일반_유효만기_재보험자별

,RINSC_CD,KICS_CRD_GRD,EFF_MAT,LIAB,PV_LIAB_LOSS_ADJ
0,121002,3,0,"5,066,410","5,037,323"
1,121005,3,1,"70,882,249","70,272,657"
2,121017,3,0,"54,318,988","54,015,896"
3,121018,3,1,"159,975,769","158,848,301"
4,121028,3,1,"9,074,872","8,993,277"
...,...,...,...,...,...
311,545024,3,1,"1,145,973","1,133,640"
312,545026,2,1,"153,784,331","152,178,200"
313,999S13,2,1,"305,038,649","300,208,021"
314,999S14,2,1,"24,079,410","23,698,086"


In [96]:
pd.options.display.float_format = '{:,.0f}'.format

# 현행추정부채
## 보험료부채
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'BOZ_CD']
일반_미경과보험료_집계 = 일반_미경과보험료_가공.groupby(KEYS)['URND_PRM'].sum().reset_index()
일반_예정보험료_집계 = 일반_예정보험료_가공.groupby(KEYS)['EPCT_PRM'].sum().reset_index()
일반_보험료부채 = 일반_미경과보험료_집계 \
    .merge(일반_예정보험료_집계, on=KEYS, how='outer') \
    .merge(일반_최종손해율, on=['RRNR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_사업비율_가공, on=['RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'], how='left') \
    .merge(일반_손해조사비율, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_할인요소[['KICS_SCEN_NO', 'PDGR_CD', 'DISC_FAC_PRM']], on=['PDGR_CD'], how='left') \
    .merge(일반_손실조정율[['KICS_SCEN_NO', 'RRNR_DVCD', 'PDGR_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_PRM']], on=['KICS_SCEN_NO', 'RRNR_DVCD', 'PDGR_CD', 'CRD_GRD'], how='left') \
    .merge(일반_유효만기[['PDGR_CD', 'EFF_MAT_PRM']], on=['PDGR_CD'], how='left') \
    .assign(URND_PRM = lambda x: x['URND_PRM'].fillna(0).astype(float)) \
    .assign(EPCT_PRM = lambda x: x['EPCT_PRM'].fillna(0).astype(float)) \
    .assign(EXP_RT = lambda x: x['EXP_RT'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_PRM = lambda x: x['LOSS_ADJ_RATE_PRM'].fillna(0).astype(float))

일반_보험료부채 = 일반_보험료부채.eval('MAINT_EXP = (URND_PRM+EPCT_PRM)*EXP_RT') \
        .eval('FNAL_LOSS = (URND_PRM+EPCT_PRM)*FNAL_LSRT') \
        .eval('LAE = FNAL_LOSS*LAE_RATIO') \
        .eval('LIAB_PRM = -EPCT_PRM+FNAL_LOSS+LAE+MAINT_EXP') \
        .eval('PV_EPCT_PRM = EPCT_PRM*DISC_FAC_PRM') \
        .eval('PV_FNAL_LOSS = FNAL_LOSS*DISC_FAC_PRM') \
        .eval('PV_LAE = LAE*DISC_FAC_PRM') \
        .eval('PV_MAINT_EXP = MAINT_EXP*DISC_FAC_PRM') \
        .eval('PV_LIAB_PRM = LIAB_PRM*DISC_FAC_PRM') \
        .eval('LOSS_ADJ_PRM = LIAB_PRM*LOSS_ADJ_RATE_PRM')
일반_보험료부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_보험료부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_보험료부채_집계 = 일반_보험료부채.groupby(['KICS_SCEN_NO', 'RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])[['PV_EPCT_PRM', 'PV_FNAL_LOSS', 'PV_LAE', 'PV_MAINT_EXP', 'PV_LIAB_PRM', 'LOSS_ADJ_PRM']].sum().reset_index()

## 준비금부채
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD', 'RINSC_CD', 'CRD_GRD', 'KICS_CRD_GRD', 'BOZ_CD']
일반_개별추산액_집계 = 일반_개별추산액_가공 \
        .groupby(KEYS)['OST_AMT'].sum().reset_index()
일반_미경과보험료_부분합 = 일반_미경과보험료_가공 \
    .eval('RRNR_DVCD_2 = RRNR_DVCD.str.replace("01|02", "04", regex=True)') \
    .assign(PDGR_CD_2 = lambda x: np.where(x['DMFR_DVCD'] == '02', '#', x['PDGR_CD'])) \
    .groupby(['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'])['URND_PRM'].sum().reset_index(name='URND_PRM_SUM')
일반_미경과보험료_비중 = 일반_미경과보험료_가공 \
    .eval('RRNR_DVCD_2 = RRNR_DVCD.str.replace("01|02", "04", regex=True)') \
    .assign(PDGR_CD_2 = lambda x: np.where(x['DMFR_DVCD'] == '02', '#', x['PDGR_CD'])) \
    .merge(일반_미경과보험료_부분합, on=['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'], how='left') \
    .eval('URND_PRM_WGT = URND_PRM/URND_PRM_SUM')
일반_IBNR_집계 = 일반_미경과보험료_비중.merge(일반_IBNR_가공.rename(columns={'RRNR_DVCD': 'RRNR_DVCD_2', 'PDGR_CD': 'PDGR_CD_2'}), on=['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'], how='left') \
    .eval('IBNR = IBNR_TOT*URND_PRM_WGT') \
    .groupby(KEYS, as_index=False)['IBNR'].sum()
일반_준비금부채 = 일반_개별추산액_집계 \
    .merge(일반_IBNR_집계, on=KEYS, how='outer') \
    .merge(일반_손해조사비율, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_할인요소[['KICS_SCEN_NO', 'PDGR_CD', 'DISC_FAC_RSV']], on=['PDGR_CD'], how='left') \
    .merge(일반_손실조정율[['KICS_SCEN_NO', 'RRNR_DVCD', 'PDGR_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_RSV']], on=['KICS_SCEN_NO', 'RRNR_DVCD', 'PDGR_CD', 'CRD_GRD'], how='left') \
    .merge(일반_유효만기[['PDGR_CD', 'EFF_MAT_RSV']], on=['PDGR_CD'], how='left') \
    .assign(OST_AMT = lambda x: x['OST_AMT'].fillna(0).astype(float)) \
    .assign(IBNR = lambda x: x['IBNR'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_RSV = lambda x: x['LOSS_ADJ_RATE_RSV'].fillna(0).astype(float)) \
    .eval('FUT_LAE = (OST_AMT*0.5+IBNR*1.0)*LAE_RATIO') \
    .eval('LIAB_RSV = OST_AMT+IBNR+FUT_LAE') \
    .eval('PV_OST_AMT = OST_AMT*DISC_FAC_RSV') \
    .eval('PV_IBNR = IBNR*DISC_FAC_RSV') \
    .eval('PV_FUT_LAE = FUT_LAE*DISC_FAC_RSV') \
    .eval('PV_LIAB_RSV = LIAB_RSV*DISC_FAC_RSV') \
    .eval('LOSS_ADJ_RSV = LIAB_RSV*LOSS_ADJ_RATE_RSV')
일반_준비금부채.loc[lambda x: x['DMFR_DVCD'] == '02', 'BOZ_CD'] = '#'
일반_준비금부채.loc[lambda x: x['RRNR_DVCD'].isin(['01', '02']), 'RRNR_DVCD'] = '04'
일반_준비금부채_집계 = 일반_준비금부채.groupby(['KICS_SCEN_NO', 'RRNR_DVCD', 'DMFR_DVCD', 'BOZ_CD'])[['PV_OST_AMT', 'PV_IBNR', 'PV_FUT_LAE', 'PV_LIAB_RSV', 'LOSS_ADJ_RSV']].sum().reset_index()

In [142]:
KEYS = ['RRNR_DVCD', 'DMFR_DVCD', 'CNTR_CATG_CD', 'BOZ_CD', 'PDGR_CD', 'CRD_GRD', 'CMSN_DVCD', 'P_NP_DVCD']

# 보험가격준비금익스포져
## 보험가격익스포져
# 일반_보험가격익스포져 = 일반_직전1년경과보험료_가공.groupby(['CNTR_CATG_CD', 'CMSN_DVCD', 'BOZ_CD', 'RRNR_DVCD', 'P_NP_DVCD'], as_index=False)[['ELP_PRM']].sum()
일반_보험가격익스포져 = 일반_출재_직전1년경과보험료_가공.groupby(KEYS, as_index=False)['ELP_PRM'].sum()

## 준비금익스포져
일반_개별추산액_집계 = 일반_개별추산액_가공.groupby(KEYS, as_index=False)['OST_AMT'].sum()
일반_미경과보험료_부분합 = 일반_미경과보험료_가공 \
    .eval('RRNR_DVCD_2 = RRNR_DVCD.str.replace("01|02", "04", regex=True)') \
    .assign(PDGR_CD_2 = lambda x: np.where(x['DMFR_DVCD'] == '02', '#', x['PDGR_CD'])) \
    .groupby(['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'])['URND_PRM'].sum().reset_index(name='URND_PRM_SUM')
일반_미경과보험료_비중 = 일반_미경과보험료_가공 \
    .eval('RRNR_DVCD_2 = RRNR_DVCD.str.replace("01|02", "04", regex=True)') \
    .assign(PDGR_CD_2 = lambda x: np.where(x['DMFR_DVCD'] == '02', '#', x['PDGR_CD'])) \
    .merge(일반_미경과보험료_부분합, on=['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'], how='left') \
    .eval('URND_PRM_WGT = URND_PRM/URND_PRM_SUM')
일반_IBNR_집계 = 일반_미경과보험료_비중.merge(일반_IBNR_가공.rename(columns={'RRNR_DVCD': 'RRNR_DVCD_2', 'PDGR_CD': 'PDGR_CD_2'}), on=['RRNR_DVCD_2', 'PDGR_CD_2', 'DMFR_DVCD'], how='left') \
    .eval('IBNR = IBNR_TOT*URND_PRM_WGT') \
    .groupby(KEYS, as_index=False)['IBNR'].sum()
일반_준비금익스포져 = 일반_개별추산액_집계.merge(일반_IBNR_집계, on=KEYS, how='outer') \
    .merge(일반_손해조사비율, on=['RRNR_DVCD', 'DMFR_DVCD', 'PDGR_CD'], how='left') \
    .merge(일반_할인요소.query('KICS_SCEN_NO == 1')[['PDGR_CD', 'DISC_FAC_RSV']], on=['PDGR_CD'], how='left') \
    .merge(일반_손실조정율.query('KICS_SCEN_NO == 1')[['RRNR_DVCD', 'PDGR_CD', 'CRD_GRD', 'LOSS_ADJ_RATE_RSV']], on=['RRNR_DVCD', 'PDGR_CD', 'CRD_GRD'], how='left') \
    .assign(OST_AMT = lambda x: x['OST_AMT'].fillna(0).astype(float)) \
    .assign(IBNR = lambda x: x['IBNR'].fillna(0).astype(float)) \
    .assign(LOSS_ADJ_RATE_RSV = lambda x: x['LOSS_ADJ_RATE_RSV'].fillna(0).astype(float))
일반_준비금익스포져 = 일반_준비금익스포져 \
    .eval('FUT_LAE = (OST_AMT*0.5+IBNR*1.0)*LAE_RATIO') \
    .eval('LIAB_RSV = OST_AMT+IBNR+FUT_LAE') \
    .eval('PV_OST_AMT = OST_AMT*DISC_FAC_RSV') \
    .eval('PV_IBNR = IBNR*DISC_FAC_RSV') \
    .eval('PV_FUT_LAE = FUT_LAE*DISC_FAC_RSV') \
    .eval('PV_LIAB_RSV = LIAB_RSV*DISC_FAC_RSV') \
    .eval('LOSS_ADJ = LIAB_RSV*LOSS_ADJ_RATE_RSV') \
    .eval('PV_LIAB_RSV_LOSS_ADJ = PV_LIAB_RSV-LOSS_ADJ')
일반_준비금익스포져_집계 = 일반_준비금익스포져.groupby(['RRNR_DVCD', 'CNTR_CATG_CD', 'BOZ_CD', 'CMSN_DVCD', 'P_NP_DVCD'], as_index=False) \
    [['PV_LIAB_RSV_LOSS_ADJ']].sum()
일반_준비금익스포져_집계

,RRNR_DVCD,CNTR_CATG_CD,BOZ_CD,CMSN_DVCD,P_NP_DVCD,PV_LIAB_RSV_LOSS_ADJ
0,01,01,A001,01,#,"20,627,457,174"
1,01,01,A001,02,#,"1,138,960,878"
2,01,01,A002,01,#,"8,055,121,830"
3,01,01,A002,02,#,"24,200,559,214"
4,01,01,A003,01,#,"24,976,754,025"
...,...,...,...,...,...,...
115,03,04,A011,02,P,"144,770,455"
116,03,06,A003,02,P,"1,727,274"
117,03,13,A002,02,P,"3,064,672"
118,03,13,A003,01,P,0


In [144]:
# 데이터 내보내기
now = datetime.now().strftime('%Y%m%d%H%M%S')
with pd.ExcelWriter(f'result/일반_현행추정부채_보험가격준비금익스포져_{now}.xlsx') as writer:
    일반_보험료부채_집계.to_excel(writer, '일반_보험료부채', index=False)
    일반_준비금부채_집계.to_excel(writer, '일반_준비금부채', index=False)
    일반_준비금익스포져_집계.to_excel(writer, '일반_준비금익스포져', index=False)
    일반_유효만기_재보험자별.to_excel(writer, '일반_유효만기_재보험자별', index=False)